### Import

---

In [1]:
import numpy as np
from note_include.elem.Ring  import Ring
from note_include.elem.RLWE  import RLWE


---

# $\textsf{RLWE}'$

* <a href="https://eprint.iacr.org/2014/816"> FHEW: Bootstrapping Homomorphic Encryption in less than a second </a>

* <a href="https://eprint.iacr.org/2020/086"> Bootstrapping in FHEW-like Cryptosystems </a>

---

앞서 RLWE 를 배우면서 우리는 동형 곱셈에 대한 여러 문제점을 확인했다. 크게 두 가지로 볼 수 있는데 다음과 같다.

1. 곱셈을 수행하면서 발생하는 노이즈의 크기가 크다.

2. 곱셈을 수행하면서 암호문의 차원이 증가하게 된다.

FHEW 에서는 이러한 문제점을 해결하기 위해 RLWE 의 변형인 RLWE' 과 RGSW 를 정의하여 동형 곱셈을 수행하게 된다.

우리는 이번에 RLWE' 에 대해서 배울 것이다.

그 전에 가젯 분해(gadget decomposition)을 이해할 필요가 있다. 이는 다음과 같이 정의된다:

>$\textbf{Definition : Gadget decomposition}$
>
>어떠한 정수 $a \in \mathbb{Z}_q$ 가 있고 base value $B$ 가 있다고 할 때 $a$ 에 대한 가젯 분해는 다음과 같이 정의된다:
>
>\begin{align*}
>    \textsf{Gadget\_decomposition\_int}(a, B) = (a_0, a_1, \dots, a_{d-1}) \in \mathbb{Z}_B^d, 
>    \text{ where } \sum_{i=0}^{d-1} a_i \cdot B^i = a \in \mathbb{Z}_q
>\end{align*}
>
>여기서 $d$ 는 $d = \lceil \log_Bq \rceil$ 이다.

코드를 통해 확인해보자.

In [2]:
def gadget_decomposition_int(v:int, B : int, d : int) -> list[int]:
    decomposed = []
    for _ in range(d):     
        remainder = v % B  
        v = v // B         
        decomposed.append(remainder)  

    return decomposed

def gadget_composition_int(vs:list[int], B : int, d : int) -> int:
    result = 0
    for i, v in enumerate(vs):     
        result += v * (B**i)

    return result 

In [3]:
q = 128
B = 2
d = int(np.ceil(np.log(q) / np.log(B)))

a = 10
decomposed_a = gadget_decomposition_int(a, B, d)
composed_a   = gadget_composition_int(decomposed_a, B, d)

print("a            : ", a)
print("Decomposed a : ", decomposed_a)
print("Composed a   : ", composed_a)

a            :  10
Decomposed a :  [0, 1, 0, 1, 0, 0, 0]
Composed a   :  10



---

즉, 단순하게 표현해서 진법변환이라고 볼 수 있다. 이는 자연스럽게 다항식으로 확장할 수 있다.

>$\textbf{Definition : Polynomial gadget decomposition}$
>
>어떠한 다항식 $\boldsymbol{a} \in \mathcal{R}_Q$ 가 있고 base value $B$ 가 있다고 할 때 $\boldsymbol{a}$ 에 대한 가젯 분해는 다음과 같이 정의된다:
>
>\begin{align*}
>    \textsf{Gadget\_decomposition}(\boldsymbol{a}, B) = (\boldsymbol{a}_0, \boldsymbol{a}_1, \dots, \boldsymbol{a}_{d-1}) \in \mathcal{R}_B^d, 
>    \text{ where } \sum_{i=0}^{d-1} \boldsymbol{a}_i \cdot B^i = \boldsymbol{a} \in \mathcal{R}_Q.
>\end{align*}
>
>여기서 $d$ 는 $d = \lceil \log_Bq \rceil$ 이다.

마찬가지로 코드를 통해 확인해보자.

In [4]:
def gadget_decomposition(poly : Ring, B : int, d : int) -> list[Ring]:
    coeffs     = np.array(poly.coeffs)
    decomposed = []

    for _ in range(d):  
        remainder = coeffs % B  
        coeffs = coeffs   // B  
        decomposed.append(Ring(poly.n, poly.q, remainder)) 

    return decomposed 

def gadget_composition(decomposed: list[Ring], B: int, modulus: int) -> Ring:
    coeffs = np.zeros(decomposed[0].n)

    for i, d in enumerate(decomposed):
        coeffs += (B**i) * np.array(d.coeffs)
    
    coeffs = [coef % modulus for coef in coeffs]
    return Ring(decomposed[0].n, decomposed[0].q, coeffs)

# 깔끔한 출력을 위한 함수
def format_ring_list(ring_list):
    return "\n".join(repr(ring) for ring in ring_list)

In [5]:
Q = 128
N = 16
B = 2
d = int(np.ceil(np.log(Q) / np.log(B)))

poly_a            = Ring(N, Q, [i for i in range(N)])
decomposed_poly_a = gadget_decomposition(poly_a, B, d)
composed_poly_a   = gadget_composition(decomposed_poly_a, B, d)

print("Polynomial a      : ", poly_a)
print("Decomposed poly a : ") 
print(format_ring_list(decomposed_poly_a))
print("Composed poly a   : ", composed_poly_a)

Polynomial a      :  (0 + 1x + 2x^2 + 3x^3 + 4x^4 + 5x^5 + 6x^6 + 7x^7 + 8x^8 + 9x^9 + 10x^10 + 11x^11 + 12x^12 + 13x^13 + 14x^14 + 15x^15 | n=16, q=128)
Decomposed poly a : 
(0 + 1x + 0x^2 + 1x^3 + 0x^4 + 1x^5 + 0x^6 + 1x^7 + 0x^8 + 1x^9 + 0x^10 + 1x^11 + 0x^12 + 1x^13 + 0x^14 + 1x^15 | n=16, q=128)
(0 + 0x + 1x^2 + 1x^3 + 0x^4 + 0x^5 + 1x^6 + 1x^7 + 0x^8 + 0x^9 + 1x^10 + 1x^11 + 0x^12 + 0x^13 + 1x^14 + 1x^15 | n=16, q=128)
(0 + 0x + 0x^2 + 0x^3 + 1x^4 + 1x^5 + 1x^6 + 1x^7 + 0x^8 + 0x^9 + 0x^10 + 0x^11 + 1x^12 + 1x^13 + 1x^14 + 1x^15 | n=16, q=128)
(0 + 0x + 0x^2 + 0x^3 + 0x^4 + 0x^5 + 0x^6 + 0x^7 + 1x^8 + 1x^9 + 1x^10 + 1x^11 + 1x^12 + 1x^13 + 1x^14 + 1x^15 | n=16, q=128)
(0 + 0x + 0x^2 + 0x^3 + 0x^4 + 0x^5 + 0x^6 + 0x^7 + 0x^8 + 0x^9 + 0x^10 + 0x^11 + 0x^12 + 0x^13 + 0x^14 + 0x^15 | n=16, q=128)
(0 + 0x + 0x^2 + 0x^3 + 0x^4 + 0x^5 + 0x^6 + 0x^7 + 0x^8 + 0x^9 + 0x^10 + 0x^11 + 0x^12 + 0x^13 + 0x^14 + 0x^15 | n=16, q=128)
(0 + 0x + 0x^2 + 0x^3 + 0x^4 + 0x^5 + 0x^6 + 0x^7 + 0x^8 + 0x^9


---

Gadget decomposition 이 갖는 의의는 다항식의 계수(혹은 정수)의 크기를 $\mathbb{Z}_B$ 로 줄일 수 있다는 것에 있다.

그럼 계수를 줄여서 얻을 수 있는 이득은 무엇이 있을까 고민해보아야 한다.

RLWE 동형 곱셈에는 크게 두가지 문제점이 있었는데, 하나는 차원의 증가였으며, 하나는 노이즈의 크기였다.

이 때, 곱하는 다항식의 계수를 줄인다는 것은 곧 노이즈가 증가하는 정도를 줄일 수 있다는 것을 의미한다.

이해를 돕기위해 RLWE 곱셈을 다시 가져와 보자.

In [6]:
# 정규분포(Gaussin distribution)를 따르는 난수 생성기
def discrete_gaussian(n, q, mean=0., std=3.2):
    coeffs = np.round(std * np.random.randn(n)) % q
    return np.array(coeffs, dtype = int)

# 균등분포(Uniform distribution)를 따르는 난수 생성기
def discrete_uniform(n, q, min=0., max=None):
    if max is None:
        max = q
    coeffs = np.random.randint(min, max, size=n)
    return np.array(coeffs, dtype = int)

# RLWE 암호화
def RLWE_Enc(m:Ring, s:Ring, std=3.2) -> tuple[Ring, Ring]:
    N = m.n
    Q = m.q
    a = Ring(N, Q, discrete_uniform(N, Q))       # Random Number
    e = Ring(N, Q, discrete_gaussian(N, Q, std=std)) # Noise

    b = a * s + m + e
    return (a, b)

# RLWE 평문 곱셈
def RLWE_Mul_Plaintext(ct:tuple[Ring, Ring], t:Ring) -> tuple[Ring, Ring]:
    a,b = ct
    return (a*t, b*t)

# RLWE 복호화
def RLWE_Dec(ct:tuple[Ring, Ring], s:Ring) -> Ring:
    a,b = ct
    msg = b - a * s

    return msg

# 유틸리티
def crange(coeffs, q):
    coeffs = np.where((coeffs >= 0) & (coeffs <= q//2), coeffs, coeffs - q)
    return coeffs

In [7]:
N = 16
Q = 128

s  = Ring(N, Q, discrete_gaussian(N, Q, std=3.2))
m  = Ring(N, Q, [i for i in range(N)])

for k in range(9):
    t = Ring(N, Q, [k+1 for _ in range(N)])
    ct = RLWE_Enc(m, s)
    ct_mul = RLWE_Mul_Plaintext(ct, t) 
    pt_mul = RLWE_Dec(ct_mul, s)
    noise  = crange(np.array((m * t - pt_mul).coeffs), Q)

    print("Polynomial with all coefficients are ", k+1, " : Noise = ", noise)

Polynomial with all coefficients are  1  : Noise =  [ 2  2  6  2  6  2 12 10 10 12 10 10  4  0 -4 -8]
Polynomial with all coefficients are  2  : Noise =  [-14  -6 -34 -26 -34 -30 -34 -30 -42 -50 -34 -30 -26  -6  -2  18]
Polynomial with all coefficients are  3  : Noise =  [-51  -9   3 -15  -9   3   3  39  39  21  27 -59 -59 -53 -29 -53]
Polynomial with all coefficients are  4  : Noise =  [-28  28  28  60  44  28  20  28 -28 -60  36  52  20  28  12  12]
Polynomial with all coefficients are  5  : Noise =  [ 10   0 -58  60  30 -30 -50 -60   0  20   0 -10 -30   0 -30 -20]
Polynomial with all coefficients are  6  : Noise =  [-48 -60   0   0  12   0 -24 -24 -36  44 -16 -60  32 -60 -48  12]
Polynomial with all coefficients are  7  : Noise =  [ 19  63  49 -21  -7  -5 -19 -19 -33 -19 -33   9 -19 -47  39  53]
Polynomial with all coefficients are  8  : Noise =  [ 24  56 -40 -40   8  40  56 -56  56  -8 -56  40 -24 -56 -40 -40]
Polynomial with all coefficients are  9  : Noise =  [ 11  47  29 -63 -63


---

위에서 확인할 수 있듯이 곱하는 다항식의 계수의 크기가 클 수록 노이즈에도 큰 값이 곱해져서 노이즈가 증가하게 된다.

이 때 gadget decomposition 을 활용하게 되면 계수의 크기를 줄일 수 있어 노이즈가 증가하는 정도를 줄일 수 있게 된다.

RLWE 에서 gadget decomposition 을 활용하기 위해서는 암호화 알고리즘과 곱셈의 정의를 약간 수정해야 한다.

이러한 RLWE의 변형을 RLWE' 이라고 칭한다.

다음과 같이 RLWE' 암호화를 정의해보도록 하자.


>$\textbf{Definition : RLWE' encryption}$
>
>메세지 $\boldsymbol{m} \in \mathcal{R}_Q$ 가 있다고 할 때 gadget decomposition 을 활용하기 위한 RLWE' 의 암호화는 다음과 같이 정의할 수 있다:
>
>\begin{align*}
>    &\textsf{RLWE'.Enc}_B(\boldsymbol{m}, \boldsymbol{s}) = (\textsf{RLWE.Enc}(B^0 \cdot \boldsymbol{m}_0), 
>    \textsf{RLWE.Enc}(B^1 \cdot \boldsymbol{m}_1), \dots, \textsf{RLWE.Enc}(B^{d-1} \cdot \boldsymbol{m}_{d-1})), \\
>    &\text{where } (\boldsymbol{m}_i)_{i < d} = \textsf{gadget\_decomposition}(\boldsymbol{m}).
>\end{align*}

코드를 통해 확인해보자.

In [8]:
def rlwep_encrypt(m:Ring, s:Ring, CC:RLWE, B:int, d:int) -> list[tuple[Ring, Ring]]:
    ctxts = []
    for i in range(d):
        ctxt = CC.encrypt((B ** i) * m, s)
        ctxts.append(ctxt)

    return ctxts

In [13]:
N = 16
Q = 128
B = 2
d = int(np.ceil(np.log(Q) / np.log(B)))

RLWE_CC = RLWE(N, Q, std=0)                 # Noiseless encryption
s, pk   = RLWE_CC.keygen()                  # secret key, public key
m       = Ring(N, Q, [1 for _ in range(N)]) # all coefficients are 1

ctxts = rlwep_encrypt(m, s, RLWE_CC, B, d)

for ct in ctxts:
    m    = RLWE_CC.decrypt(ct, s)
    print(m)

(1 + 1x + 1x^2 + 1x^3 + 1x^4 + 1x^5 + 1x^6 + 1x^7 + 1x^8 + 1x^9 + 1x^10 + 1x^11 + 1x^12 + 1x^13 + 1x^14 + 1x^15 | n=16, q=128)
(2 + 2x + 2x^2 + 2x^3 + 2x^4 + 2x^5 + 2x^6 + 2x^7 + 2x^8 + 2x^9 + 2x^10 + 2x^11 + 2x^12 + 2x^13 + 2x^14 + 2x^15 | n=16, q=128)
(4 + 4x + 4x^2 + 4x^3 + 4x^4 + 4x^5 + 4x^6 + 4x^7 + 4x^8 + 4x^9 + 4x^10 + 4x^11 + 4x^12 + 4x^13 + 4x^14 + 4x^15 | n=16, q=128)
(8 + 8x + 8x^2 + 8x^3 + 8x^4 + 8x^5 + 8x^6 + 8x^7 + 8x^8 + 8x^9 + 8x^10 + 8x^11 + 8x^12 + 8x^13 + 8x^14 + 8x^15 | n=16, q=128)
(16 + 16x + 16x^2 + 16x^3 + 16x^4 + 16x^5 + 16x^6 + 16x^7 + 16x^8 + 16x^9 + 16x^10 + 16x^11 + 16x^12 + 16x^13 + 16x^14 + 16x^15 | n=16, q=128)
(32 + 32x + 32x^2 + 32x^3 + 32x^4 + 32x^5 + 32x^6 + 32x^7 + 32x^8 + 32x^9 + 32x^10 + 32x^11 + 32x^12 + 32x^13 + 32x^14 + 32x^15 | n=16, q=128)
(64 + 64x + 64x^2 + 64x^3 + 64x^4 + 64x^5 + 64x^6 + 64x^7 + 64x^8 + 64x^9 + 64x^10 + 64x^11 + 64x^12 + 64x^13 + 64x^14 + 64x^15 | n=16, q=128)



---

변형된 암호화에 따라서 연산의 정의 또한 바꾸어주어야 한다. RLWE' 의 곱셈은 다음과 같이 정의한다:

>$\textbf{Definition : RLWE' plaintext multiplication}$
>
>RLWE' 암호문 $\textsf{ct} = (\textsf{ct}_i)_{i < d} \in \mathcal{R}_Q^{2 \times d}$ 와 평문 다항식 $\boldsymbol{t} \in \mathcal{R}_Q$ 가 있다고 하자.
>
>이 때 RLWE' 과 평문 다항식 간의 곱셈$(\ast)$은 다음과 같이 정의한다:
>
>\begin{align*}
>    &(\ast) : \text{RLWE}' \ast \mathcal{R}_Q \rightarrow \text{RLWE} \\
>    &\textsf{ct} \ast \boldsymbol{t} = \sum_{i=0}^{d-1} \boldsymbol{t}_i \circ \textsf{ct}_i, \text{ where } (\boldsymbol{t}_i)_{i<d} 
>    = \textsf{gadget\_decomposition}(\boldsymbol{t})
>\end{align*}
>
>여기서 $(\circ)$ 연산은 RLWE 암호문과 평문 다항식 간의 곱셈을 나타낸다.

코드를 통해 확인해보자.

In [16]:
def rlwep_mult_poly(CC:RLWE, ctxts:list[tuple[Ring, Ring]], poly:Ring, B, d) -> tuple[Ring, Ring]:
    q = poly.q
    n = poly.n

    decomposed_polys = gadget_decomposition(poly, B, d)
    zero_coeffs = np.zeros(n)
    zero_poly   = Ring(n, q, zero_coeffs)
    result      = [zero_poly, zero_poly]
    
    for ctxt, d_poly in zip(ctxts, decomposed_polys):
        if all(c == 0 for c in d_poly.coeffs):continue
        tmp = result
        result = CC.add_ctxt_ctxt(tmp, CC.mult_ring_ptxt(ctxt, d_poly))
    
    return result

In [34]:
N = 16
Q = 128
B = 2
d = int(np.ceil(np.log(Q) / np.log(B)))

RLWE_CC = RLWE(N, Q, std=3.2)                 # Noiseless encryption
s, pk   = RLWE_CC.keygen()                  # secret key, public key
m       = Ring(N, Q, [1 for _ in range(N)]) # all coefficients are 1
t       = Ring(N, Q, [10 for _ in range(N)])

ctxts = rlwep_encrypt(m, s, RLWE_CC, B, d)
ct    = rlwep_mult_poly(RLWE_CC, ctxts, t, B, d)
ptxt  = RLWE_CC.decrypt(ct, s)
noise  = crange(np.array((m * t - ptxt).coeffs), Q)

print("Result           : ", ptxt)
print("(Ideal) Result   : ", m * t)
print("RLWEp mult Noise : ", noise)

Result           :  (114 + 122x + 4x^2 + 16x^3 + 40x^4 + 50x^5 + 46x^6 + 58x^7 + 106x^8 + 2x^9 + 24x^10 + 48x^11 + 84x^12 + 114x^13 + 6x^14 + 22x^15 | n=16, q=128)
(Ideal) Result   :  (116 + 8x + 28x^2 + 48x^3 + 68x^4 + 88x^5 + 108x^6 + 0x^7 + 20x^8 + 40x^9 + 60x^10 + 80x^11 + 100x^12 + 120x^13 + 12x^14 + 32x^15 | n=16, q=128)
RLWEp mult Noise :  [  2  14  24  32  28  38  62 -58  42  38  36  32  16   6   6  10]



---

그럼 이번에는 RLWE 곱셈과 노이즈 정도의 차이를 비교해보자.

In [46]:
N = 16
Q = 128
B = 2
d = int(np.ceil(np.log(Q) / np.log(B)))

RLWE_CC = RLWE(N, Q, std=3.2)                 # Noiseless encryption
s, pk   = RLWE_CC.keygen()                  # secret key, public key
m       = Ring(N, Q, [1 for _ in range(N)]) # all coefficients are 1
t       = Ring(N, Q, [10 for _ in range(N)])

rlwe_ct     = RLWE_CC.encrypt(m, s)
rlwe_ct_mul = RLWE_CC.mult_ring_ptxt(rlwe_ct, t)
rlwe_ptxt   = RLWE_CC.decrypt(rlwe_ct_mul, s)
rlwe_noise  = crange(np.array((m * t - rlwe_ptxt).coeffs), Q)

rlwep_ct     = rlwep_encrypt(m, s, RLWE_CC, B, d)
rlwep_ct_mul = rlwep_mult_poly(RLWE_CC, rlwep_ct, t, B, d)
rlwep_ptxt   = RLWE_CC.decrypt(rlwep_ct_mul, s)
rlwep_noise  = crange(np.array((m * t - rlwep_ptxt).coeffs), Q)

print("RLWE  mult Noise : ", rlwe_noise)
print("RLWEp mult Noise : ", rlwep_noise)

RLWE  mult Noise :  [-52   8   8  28 -40   0  20 -48  32  44  24   4 -44  64 -56  52]
RLWEp mult Noise :  [ -5 -15  -1  -3   7   1   5   9   3  19  19   5  -7  -5  -9   5]



---

위에서 확인할 수 있듯 일반적인 RLWE 곱셈보다 RLWE' 곱셈을 수행하는 것이 노이즈를 관리하는 측면에서 더욱 효율적이라는 것을 알 수 있다.

$B$ 를 낮출수록 계수의 크기 또한 작아져 노이즈를 더 효울적으로 관리할 수 있으나, 반대로 다항식의 수(즉, 암호문의 수)가 늘어나서 연산을 수행하기 위한 비용이 늘어난다는 특징이 있다.

---

## Summary

곱셈을 효율적으로 수행하기 위해 정의된 RLWE 의 변형, RLWE' 에 대해서 알아보았다.

Gadget decomposition 을 통해 계수(혹은 정수)의 크기를 낮추어 노이즈를 관리하는 측면에서 일반적인 RLWE 곱셈보다 더욱 효율적이라는 것을 확인할 수 있었다.

RLWE' 은 평문곱셈을 효율적으로 수행하기 위해 고안되었다.

이 다음에는 RLWE' 을 활용하여 동형 곱셈을 효율적으로 처리하는 RGSW 에 대해서 알아볼 것이다.

---

## Code implementation

`note_include/elem/RLWEp.py` 를 확인하면 위에서 정의한 연산들이 있는 RLWE' 구현을 확인할 수 있다.